# Machine Larning Project

Article: *Multi-Task Deep Neural Networks for Natural Language Understanding*

<a href=https://arxiv.org/abs/1901.11504> https://arxiv.org/abs/1901.11504</a>

## Dowload Data

In [1]:
#!cd ..
#!python scripts/download_glue_data.py --data_dir data --tasks all


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc

